In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import GaussianNB
from skmultiflow.trees import HoeffdingTreeClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from skmultiflow.drift_detection import DDM
from skmultiflow.drift_detection import EDDM
from skmultiflow.drift_detection import ADWIN
from skmultiflow.drift_detection import HDDM_A
from skmultiflow.drift_detection import HDDM_W

from tqdm import tqdm
import itertools
import random
import seaborn as sns
from sklearn.decomposition import PCA

from scipy import stats
from scipy.stats import mannwhitneyu
from math import log2
from sklearn.metrics.pairwise import manhattan_distances
from scipy.spatial.distance import chebyshev
from scipy.spatial.distance import kulsinski
from scipy.spatial.distance import cosine
from scipy.spatial.distance import sqeuclidean

from sklearn.preprocessing import MinMaxScaler

import warnings
from IPython.display import clear_output

In [2]:
def kl_divergence(p, q):
    return sum(p[0][i] * log2(p[0][i]/q[0][i]) for i in range(len(p[0])))

In [3]:
def bhattacharyya(p, q):
    return -np.log(sum(np.square(p[0][i]*1.0*q[0][i]) for i in range(len(p[0]))))

In [4]:
def bootstrapping(X_train, bootstrapping_samples, pca=None):
    # Extract distributions of 50 random training subsets
    distributions_bootstrapping = []
    for i in range(0, bootstrapping_samples):
        # generate random number between 0 and length_of_train-length_of_test in order to have a sample of size length_of_test
        rand = random.randint(0,len(X_train)-length_of_test)
        # extract the distribution from the training samples random number:random number + length_of_test by means of histogram
        if pca is not None:
            bootstrapping_input = pca.transform(X_train[rand:rand+length_of_test])
        else:
            bootstrapping_input = X_train[rand:rand+length_of_test]
        dist = sns.distplot(bootstrapping_input).get_lines()[0].get_data()[1]
        # store distribution
        distributions_bootstrapping.append(dist)
        plt.close()
    return distributions_bootstrapping

In [5]:
def kdqtrees_bootstrapping(distrib_X_train, distrib_X_test, distributions_bootstrapping, distance):
    
    # option to choose the similarity distance
    
    if(distance == 'kl_divergence'):
        similarity_metric = kl_divergence
    elif(distance == 'manhattan'):
        similarity_metric = manhattan_distances
    elif(distance == 'chebyshev'):
        similarity_metric = chebyshev
    elif(distance == 'kulsinski'):
        similarity_metric = kulsinski
    elif(distance == 'cosine'):
        similarity_metric = cosine
    elif(distance == 'squared_euclidean'):
        similarity_metric = sqeuclidean
    elif(distance == 'bhattacharyya'):
        similarity_metric = bhattacharyya
    
    # Bootstrapping technique to define critical region
    
    dist_bootstrapping = []
    
    # Calculate similarity distance between training set and each of the 50 training subsets
    for i in range(0, len(distributions_bootstrapping)):
        dist_bootstrapping.append(similarity_metric([distrib_X_train], [distributions_bootstrapping[i]]))
    
    
    # Define Critical Region
    critical_region = np.max(dist_bootstrapping)
    
    # Detect Drifts between train and test
    
    drift_batch = None
    
    # Calculate distance between train and test distributions
    
    similarity_metric_train_test = similarity_metric([distrib_X_train], [distrib_X_test])
    
    if(similarity_metric_train_test>critical_region):
        print(distance + " : Drift Detected" )
        return 1
    else:
        return 0

In [6]:
def compute_metric_false_positive(array_batches, drift_start):
        
    if(len(array_batches)>0): 
        
        if(len([x for x in array_batches if x<drift_start])>0):
            return(len([x for x in array_batches if x<drift_start])/drift_start)
        else:
            return 0
        
    else:
        return 'nothing_detected'

In [7]:
def ede_drift_detector(distrib_X_train, distrib_X_test, statistical_test):

    if(statistical_test == 'ks'):
        stat_test = stats.kstest
    elif(statistical_test == 'mw'):
        stat_test = stats.mannwhitneyu
    
    v, p = stat_test(distrib_X_train, distrib_X_test)
        
    
    if(p<=0.05):
        print("Reject Null Hypothesis according to KS statistical test")
        return 1
    else:
        return 0

In [8]:
def compute_metric_latency(array_batches, no_batches_with_drift, drift_start):
    
    if(len(array_batches)>0):  
        #print(np.array(array_pr)>=drift_type_start)
        #print(np.argwhere(np.array(array_pr)>=drift_type_start).size==0)
        
        if(np.argwhere(np.array(array_batches)>=drift_start).size==0):
            latency_score = 'nothing_detected' 
        else:
            batch_drift_detected = array_batches[np.argwhere(np.array(array_batches)>=drift_start)[0][0]]
            latency_score = (batch_drift_detected - drift_start)/no_batches_with_drift
        return latency_score
    else:
        return 'nothing_detected'

In [9]:
def drift_detect_function(drift_detector, y_true, y_pred, no_batches):
    batches_with_drift = []
    for i in range(0, no_batches):
        label_difference = abs(y_true[i] - y_pred[i])
        #print(label_difference)
        
        for j in range(0, len(label_difference)):
            drift_detector.add_element(list(label_difference)[j])

            if(drift_detector.detected_change()):
                batches_with_drift.append(i)
    return batches_with_drift

In [10]:
# ADWIN has: 1 for correct prediction and 0 for incorrect prediction
def drift_detect_function_ADWIN(drift_detector, y_true, y_pred, no_batches):
    
    batches_with_drift = []
    for i in range(0, no_batches):
        label_difference = abs(y_true[i] - y_pred[i])
        label_difference_final = 1 - label_difference
    
        for j in range(0, len(label_difference_final)):
            drift_detector.add_element(list(label_difference_final)[j])
            if(drift_detector.detected_change()):
                batches_with_drift.append(i)
    return batches_with_drift

In [11]:
warnings.filterwarnings('ignore')

# Airlines Dataset

In [12]:
data_airlines = arff.loadarff('../airlines.arff')
df_airlines = pd.DataFrame(data_airlines[0])


### Encode Labels

In [13]:
labelencoder = LabelEncoder()
df_airlines['Delay'] = labelencoder.fit_transform(df_airlines['Delay'])


# Feature Analysis

### Feature Cardinality Analysis

In [14]:
len(df_airlines.Airline.value_counts())

18

In [15]:
len(df_airlines.AirportFrom.value_counts())

293

In [16]:
len(df_airlines.AirportTo.value_counts())

293

In [17]:
#df_airlines[(df_airlines.Flight == 269.0) & (df_airlines.Airline == list(df_airlines.Airline)[0])]

Issue: Airport From and Airport To have 293 different values each, which makes them high cardinal features. This results in adding almost 600 new features through One Hot Encoding, which could make the model suffer from the curse of dimensionality. 

Solution: we can remove AirportTo and AirportFrom, since the flight number + airline is the same for each source and destination. However, the flight number can be the same for multiple airlines, so Airline should not be removed.

### Remove high cardinality features

In [18]:
df_airlines = df_airlines.drop(['AirportFrom', 'AirportTo'], axis = 1)

### One Hot Encoding on Airline

In [19]:
one_hot_encoded_data = pd.get_dummies(df_airlines, columns = ['Airline'])
df = one_hot_encoded_data


### Data Scaling

In [20]:
scaler = MinMaxScaler(feature_range = (0,1))

In [21]:
scaler.fit(df.drop(['DayOfWeek', 'Delay'], axis = 1))
df_scale = scaler.transform(df.drop(['DayOfWeek', 'Delay'], axis = 1))
df_features_train_test = pd.DataFrame(df_scale)

In [22]:
df_labels_train_test = df['Delay']

## Week

In [23]:
# the numbers here correspond to the exact moment when the second week starts and 
# the exact moment for each day of the week

In [24]:
# including Day 1 & 2 of the second week (Monday and Tuesday) into the training set
df_train_1 = df_features_train_test[85190:120388] 
df_labels_train_1 = df_labels_train_test[85190:120388]


In [25]:
# extract data corresponding to Wednesday
df_test_wed = df_features_train_test[120388:137920]
df_labels_test_wed = df_labels_train_test[120388:137920]


In [26]:
# extract data corresponding to Thursday
df_test_thu = df_features_train_test[137920:155976]
df_labels_test_thu = df_labels_train_test[137920:155976]


In [27]:
# extract data corresponding to Friday
df_test_fri = df_features_train_test[155976:174115]
df_labels_test_fri = df_labels_train_test[155976:174115]


In [28]:
# extract data corresponding to Saturday
df_test_sat = df_features_train_test[174115:188415]
df_labels_test_sat = df_labels_train_test[174115:188415]


In [29]:
# extract data corresponding to Sunday
df_test_sun = df_features_train_test[188415:205551]
df_labels_test_sun = df_labels_train_test[188415:205551]


In [30]:
df_test_total = [df_test_wed, df_test_thu, df_test_fri, df_test_sat, df_test_sun]
df_test_labels_total = [df_labels_test_wed, df_labels_test_thu, df_labels_test_fri, df_labels_test_sat, df_labels_test_sun]


# EB Detectors

### Train Classifiers

In [31]:
gnb = GaussianNB()
naive_bayes = gnb.fit(df_train_1, df_labels_train_1)

ht = HoeffdingTreeClassifier()
hoeffding_tree = ht.fit(np.array(df_train_1), np.array(df_labels_train_1))

adb = AdaBoostClassifier(n_estimators= 100, random_state=0)
adaboost = adb.fit(df_train_1, df_labels_train_1)

xgb = XGBClassifier()
xgboost = xgb.fit(df_train_1, df_labels_train_1)

lgb = LGBMClassifier()
lightgbm = lgb.fit(df_train_1, df_labels_train_1)

[19:05:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


### Get Predictions

In [32]:
y_pred_naivebayes = []
y_pred_hoftree = []
y_pred_adaboost = []
y_pred_xgboost = []
y_pred_lightgbm = []

for i in tqdm(range(0, len(df_test_total))):
    # naive bayes
    y_pred_nb = naive_bayes.predict(df_test_total[i])
    y_pred_naivebayes.append(y_pred_nb)

    # hoeffding tree
    y_pred_ht = hoeffding_tree.predict(np.array(df_test_total[i]))
    y_pred_hoftree.append(y_pred_ht)
    # adaboost
    y_pred_adb = adaboost.predict(df_test_total[i])
    y_pred_adaboost.append(y_pred_adb)
    # xgboost
    y_pred_xgb = xgboost.predict(df_test_total[i])
    y_pred_xgboost.append(y_pred_xgb)
    # lightgbm
    y_pred_lgbm = lightgbm.predict(df_test_total[i])
    y_pred_lightgbm.append(y_pred_lgbm)

100%|██████████| 5/5 [00:04<00:00,  1.09it/s]


In [33]:
y_pred_dict = {'NaiveBayes': y_pred_naivebayes, 
                   'HoeffdingTree' : y_pred_hoftree, 
                   'Adaboost' : y_pred_adaboost,
                   'XGBoost' : y_pred_xgboost,
                   'LightGBM' : y_pred_lightgbm}

### Detect Drifts

In [34]:
no_batches = len(df_test_labels_total)

In [35]:
classifier_list = []
detector_list = []
batches_detected = []

for i in range(0, len(list(y_pred_dict.keys()))):
    classifier_list.append(list(y_pred_dict.keys())[i])
    
    detector_list.append('DDM')
    batches_detected.append(drift_detect_function(DDM(), df_test_labels_total, list(y_pred_dict.values())[i], no_batches))
    
    detector_list.append('EDDM')
    batches_detected.append(drift_detect_function(EDDM(), df_test_labels_total, list(y_pred_dict.values())[i], no_batches))
    
    detector_list.append('ADWIN')
    batches_detected.append(drift_detect_function(ADWIN(), df_test_labels_total, list(y_pred_dict.values())[i], no_batches))
    
    detector_list.append('HDDM_A')
    batches_detected.append(drift_detect_function(HDDM_A(), df_test_labels_total, list(y_pred_dict.values())[i], no_batches))
    
    detector_list.append('HDDM_W')
    batches_detected.append(drift_detect_function(HDDM_W(), df_test_labels_total, list(y_pred_dict.values())[i], no_batches))

batches_detected_clean = []
for i in range(0, len(batches_detected)):
    batches_detected_clean.append(list(dict.fromkeys(batches_detected[i])))

classifier_list = list(itertools.chain.from_iterable(itertools.repeat(x, 5) for x in classifier_list))


In [36]:
df_airlines_eb_detectors = pd.DataFrame(list(zip(classifier_list, detector_list, batches_detected_clean)), columns =['Classifier', 'Detector', 'Batches_Detected'])


# DDB Detectors

Reference Data

In [37]:
#df_train_1

### Distribution of Reference Data

In [38]:
pca = PCA(n_components = 0.999)
pca.fit(df_train_1)

df_train_1_pca = pca.transform(df_train_1)

In [39]:
distribution_train = sns.distplot(df_train_1).get_lines()[0].get_data()[1]
plt.close()

distribution_train_pca = sns.distplot(df_train_1_pca).get_lines()[0].get_data()[1]
plt.close()

### Distribution Bootstrapping

In [40]:
# length_of_test should be around 15000 (best compromise between all testing sets)
length_of_test = 15000
distributions_bootstrapping = bootstrapping(df_train_1, bootstrapping_samples=50)

In [41]:
distributions_bootstrapping_pca = bootstrapping(df_train_1, bootstrapping_samples=50, pca=pca)

In [42]:
distributions_test = []
distributions_test_pca = []

for i in range(0, len(df_test_total)):
    dist_test = sns.distplot(df_test_total[i]).get_lines()[0].get_data()[1]
    plt.close()
    
    dist_test_pca = sns.distplot(pca.transform(df_test_total[i])).get_lines()[0].get_data()[1]
    plt.close()
    
    distributions_test.append(dist_test)
    distributions_test_pca.append(dist_test_pca)

## EDE

In [43]:
detected_drifts_ks = []
detected_drifts_mw = []
for i in range(0, len(distributions_test)):
    if(ede_drift_detector(distribution_train, distributions_test[i], 'ks')==1):
        detected_drifts_ks.append(i)
    if(ede_drift_detector(distribution_train, distributions_test[i], 'mw')==1):
        detected_drifts_mw.append(i)
    clear_output(wait=True)

test = ['ks', 'mw']
drifts = [detected_drifts_ks, detected_drifts_mw]

In [44]:
df_ede_bootstrapping = pd.DataFrame({'detector' : 'ede',
                                    'distance/test': test, 
                                    'Batches_Detected': drifts})

In [45]:
# df_ede_bootstrapping

## kdqTrees

In [46]:
kl_drift = []
mh_drift = []
cbs_drift = []
ksnk_drift = []
csn_drift = []
sqe_drift = []
bct_drift = []

for i in range(0, len(distributions_test)):
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'kl_divergence') == 1):
        kl_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'manhattan') == 1):
        mh_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'chebyshev') == 1):
        cbs_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'kulsinski') == 1):
        ksnk_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'cosine') == 1):
        csn_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'squared_euclidean') == 1):
        sqe_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train, distributions_test[i], distributions_bootstrapping, 'bhattacharyya') == 1):
        bct_drift.append(i)
    clear_output(wait=True)

distances = ['kl_div', 'manhattan', 'chebysev', 'kulsinski', 'cosine', 'sq_euclid', 'batthacrya']
drifts = [kl_drift, mh_drift, cbs_drift, ksnk_drift, csn_drift, sqe_drift, bct_drift]
df_kdqtrees_bootstrapping = pd.DataFrame({'detector' : 'kdqTrees',
                                        'distance/test': distances, 
                                        'Batches_Detected': drifts
                                })

In [47]:
# df_kdqtrees_bootstrapping

## PCA-kdq

In [48]:
kl_drift = []
mh_drift = []
cbs_drift = []
ksnk_drift = []
csn_drift = []
sqe_drift = []
bct_drift = []

for i in range(0, len(distributions_test_pca)):

    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'kl_divergence') == 1):
        kl_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'manhattan') == 1):
        mh_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'chebyshev') == 1):
        cbs_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'kulsinski') == 1):
        ksnk_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'cosine') == 1):
        csn_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'squared_euclidean') == 1):
        sqe_drift.append(i)
    if(kdqtrees_bootstrapping(distribution_train_pca, distributions_test_pca[i], distributions_bootstrapping_pca, 'bhattacharyya') == 1):
        bct_drift.append(i)
    clear_output(wait=True)

distances = ['kl_div', 'manhattan', 'chebysev', 'kulsinski', 'cosine', 'sq_euclid', 'batthacrya']
drifts = [kl_drift, mh_drift, cbs_drift, ksnk_drift, csn_drift, sqe_drift, bct_drift]
df_pca_bootstrapping = pd.DataFrame({'detector' : 'pca',
                                     'distance/test': distances, 
                                     'Batches_Detected': drifts
                                })

merged_results = pd.concat([df_ede_bootstrapping, df_kdqtrees_bootstrapping, df_pca_bootstrapping])
merged_results = merged_results.reset_index(drop=True)



In [49]:
df_airlines_ddb_detectors = pd.concat([df_ede_bootstrapping, df_kdqtrees_bootstrapping, df_pca_bootstrapping])

# Calculate metrics

In [50]:
no_batches_with_drift = 3 # there are 3 testing batches with drift corresponding to Friday, Saturday and Sunday
drift_start = 2 # drift starts at batch 2, corresponding to Friday

In [51]:
latency_erb = []
fpr_erb = []

latency_ddb = []
fpr_ddb = []

for i in range(0, len(list(df_airlines_eb_detectors.Batches_Detected))):
    latency_erb.append(compute_metric_latency(list(df_airlines_eb_detectors.Batches_Detected)[i], no_batches_with_drift, drift_start))
    fpr_erb.append(compute_metric_false_positive(list(df_airlines_eb_detectors.Batches_Detected)[i], drift_start))

for i in range(0, len(list(df_airlines_ddb_detectors.Batches_Detected))):
    latency_ddb.append(compute_metric_latency(list(df_airlines_ddb_detectors.Batches_Detected)[i], no_batches_with_drift, drift_start))
    fpr_ddb.append(compute_metric_false_positive(list(df_airlines_ddb_detectors.Batches_Detected)[i], drift_start))

In [52]:
df_airlines_eb_detectors['latency'] = latency_erb
df_airlines_eb_detectors['fpr'] = fpr_erb
df_airlines_eb_detectors

,Classifier,Detector,Batches_Detected,latency,fpr
0,NaiveBayes,DDM,"[0, 4]",0.666667,0.5
1,NaiveBayes,EDDM,[0],nothing_detected,0.5
2,NaiveBayes,ADWIN,"[0, 1, 2, 3, 4]",0.0,1.0
3,NaiveBayes,HDDM_A,"[0, 1, 2, 3, 4]",0.0,1.0
4,NaiveBayes,HDDM_W,"[0, 1, 2, 3, 4]",0.0,1.0
5,HoeffdingTree,DDM,[0],nothing_detected,0.5
6,HoeffdingTree,EDDM,"[0, 1, 2, 3, 4]",0.0,1.0
7,HoeffdingTree,ADWIN,"[0, 1, 2, 3, 4]",0.0,1.0
8,HoeffdingTree,HDDM_A,"[0, 1, 2, 3, 4]",0.0,1.0
9,HoeffdingTree,HDDM_W,"[0, 1, 2, 3, 4]",0.0,1.0


In [53]:
df_airlines_ddb_detectors['latency'] = latency_ddb
df_airlines_ddb_detectors['fpr'] = fpr_ddb
df_airlines_ddb_detectors

,detector,distance/test,Batches_Detected,latency,fpr
0,ede,ks,[],nothing_detected,nothing_detected
1,ede,mw,[],nothing_detected,nothing_detected
0,kdqTrees,kl_div,[],nothing_detected,nothing_detected
1,kdqTrees,manhattan,[],nothing_detected,nothing_detected
2,kdqTrees,chebysev,[],nothing_detected,nothing_detected
3,kdqTrees,kulsinski,[],nothing_detected,nothing_detected
4,kdqTrees,cosine,[],nothing_detected,nothing_detected
5,kdqTrees,sq_euclid,[],nothing_detected,nothing_detected
6,kdqTrees,batthacrya,[],nothing_detected,nothing_detected
0,pca,kl_div,[],nothing_detected,nothing_detected
